## SISSO descriptors application to a 10,000 random sampling in the system MoNbTaVW

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import os
import multiprocessing
import time
import pickle

In [2]:
import pandas as pd
import numpy as np
from itertools import combinations

### SISSO 3D ####
## the nature of the descriptor makes it difficult to work without a database of unary features
## The idea is for the descriptor to take the compositions of the predefined [Mn,Nb,Ta,V,W] vector
## The object only needs to be initialized once and then can be called

class SISSO:
    def __init__(self, csv_una,string_of,d):
        self.df_una=pd.read_csv(csv_una)
        self.df_una.columns =[column.replace(" ", "_") for column in self.df_una.columns]
        self.df_una.columns =[column.replace("'", "_") for column in self.df_una.columns]
        self.df_una.columns =[column.replace("(", "_") for column in self.df_una.columns]
        self.df_una.columns =[column.replace(")", "_") for column in self.df_una.columns]
        self.df_una.columns =[column.replace("/", "_") for column in self.df_una.columns]
        self.df_una.columns=self.df_una.columns.str.lower()
        self.df_una=self.df_una.set_index('symbol',drop=True)
        self.org_ele=['Mo','Nb','Ta','V','W']   #they have to be in alphabetical order 
        self.v_wo_df=[]
        
        features=self.df_una.columns
        raw=[]
        pattern = str(d)+"D descriptor"
        #print(features)
        with open(string_of) as file:
            all_lines = file.readlines()
            for current_line_no, current_line in enumerate(all_lines):
                if pattern in current_line:
                    for k in range(d+2):
                        raw.append(all_lines[current_line_no+3+k].split(':')[1])
                    break
        #print(raw)
        v_lines=[0]*d
        used_feat=[]
        for i in range(d):
            v_lines[i]=raw[i]
            v_lines[i]=v_lines[i][1:-2]
            self.v_wo_df.append(v_lines[i][:])
            v_lines[i]=v_lines[i].replace('^','**')
            v_lines[i]=v_lines[i].replace('exp','np.exp')
            v_lines[i]=v_lines[i].replace('sqrt','np.sqrt')
            v_lines[i]=v_lines[i].replace('log','np.log')
            
            for f in features:
                #print(f)

                if f in raw[i]:
                    if f not in used_feat:
                        used_feat.append(f)


                    v_lines[i]=v_lines[i].replace(f+'_avg','df[\''+f+'_avg'+'\'].values[0]')
                    v_lines[i]=v_lines[i].replace(f+'_red','df[\''+f+'_red'+'\'].values[0]')
                    v_lines[i]=v_lines[i].replace(f+'_diff','df[\''+f+'_diff'+'\'].values[0]')
        self._used_feat=used_feat
        self.a0=float(raw[-1])
        self.a=[float(i) for i in raw[-2].split()]
        #print(self.a0)
        #print(self.a)
        #print(v_lines)
        #print(used_feat)
        self.v_lines=v_lines
        self.features=features

            
    def _red_and_diff(self,ele, stoich, feature):
        df_una=self.df_una
        num_els=len(ele)
        denom = (num_els - 1) * np.sum(stoich)

        values=np.array([df_una.loc[ei,feature] for ei in ele])
        pairs = list(combinations(ele, 2))
        pair_red_lst = []
        pair_diff_lst = []
        for i in range(len(pairs)):
            first_elem = ele.index(pairs[i][0])
            second_elem = ele.index(pairs[i][1])
            pair_coeff = stoich[first_elem] + stoich[second_elem]
            pair_prod = values[first_elem] * values[second_elem]
            pair_sum = values[first_elem] + values[second_elem]

            pair_sub = np.abs(values[first_elem] - values[second_elem])

            pair_red = pair_coeff * pair_prod / pair_sum
            pair_diff = pair_coeff * pair_sub

            pair_red_lst.append(pair_red)
            pair_diff_lst.append(pair_diff)
        return np.sum(pair_red_lst) / denom , np.sum(pair_diff_lst) / denom

    def _only_avg(self,ele,stoich, feature):
        df_una=self.df_una
        
        values=np.array([df_una.loc[ei,feature] for ei in ele])
        average = np.average(values, weights=stoich)
        return average
    def _ult_eq(self,x):
        
        df=pd.DataFrame(index=[0])
        sisso_feat=self.features
        ele=[self.org_ele[i] for i in np.where(x>0)[0]]
        x=x[np.where(x>0)[0]]
        #print(x)
        for ss in sisso_feat:
            df[ss+'_avg']=self._only_avg(ele,x,ss)
            df[ss+'_red'],df[ss+'_diff']=self._red_and_diff(ele,x,ss)
        suma=self.a0
        
        for i,c in enumerate(self.a):
            suma=suma+self.a[i]*eval(self.v_lines[i])
        return suma
        
    def ult_eq(self,x):
    #### Only the SISSO called features are being calculated to lower the computation cost
    ####
        x=np.array(x)
        if len(x.shape)>1:
            return np.apply_along_axis(self._ult_eq, 1, x)
        else:
            return self._ult_eq(x)
        
        
    def print_formula_latex(self,ff,fl):
        #print(self.a0)
        d=len(self.v_lines)
        fchange=[0]*d
        for i,c in enumerate(self.a):
            fchange[i]=self.v_wo_df[i][:]
            for f,l in zip(ff,fl):
                fchange[i]=fchange[i].replace(f,l)
            fchange[i]=fchange[i].replace('log','\ln')
            fchange[i]=fchange[i].replace('*',' ')
            
        ws="{:.2E}".format(self.a0).replace('E','\\times 10 ^')
        dummy=ws.split('^')[1]
        intdummy=int(dummy)
        ws=ws.replace(dummy,'{'+str(intdummy)+'}')
        
        final_str='$\\begin{array}{c}\n'+ws[:]
        for i in range(len(fchange)):
            
            
            
            ws="{:.2E}".format(self.a[i]).replace('E','\\times 10 ^')
            dummy=ws.split('^')[1]
            intdummy=int(dummy)
            ws=ws.replace(dummy,'{'+str(intdummy)+'}')
            
            if self.a[i]<0:
                final_str=final_str+ws+fchange[i]+'\\\\\n'
            else:
                final_str=final_str+"+"+ws+fchange[i]+'\\\\\n'
        final_str=final_str+'\end{array}$'
        print(final_str)

In [3]:
data_f='../data/'
res_f='../results/'

In [4]:
una_csv=data_f+'UNARIES_NINE.csv'

## get all SISSO models 

In [7]:
sK = SISSO(una_csv,data_f+'fromHPRC/K.avgvar00/dirUNIQUE/SISSO.out',3)
sG= SISSO(una_csv,data_f+'fromHPRC/G.avgvar00/dirUNIQUE/SISSO.out',3)
sE= SISSO(una_csv,data_f+'fromHPRC/E.avgvar00/dirUNIQUE/SISSO.out',3)
sC11= SISSO(una_csv,data_f+'fromHPRC/C_11.avgvar00/dirUNIQUE/SISSO.out',3)
sC12= SISSO(una_csv,data_f+'fromHPRC/C_12.avgvar00/dirUNIQUE/SISSO.out',3)
sC44= SISSO(una_csv,data_f+'fromHPRC/C_44.avgvar00/dirUNIQUE/SISSO.out',3)
sv= SISSO(una_csv,data_f+'fromHPRC/v.avgvar00/dirUNIQUE/SISSO.out',3)
sGK= SISSO(una_csv,data_f+'fromHPRC/GK.avgvar00/dirUNIQUE/SISSO.out',3)
scauchy = SISSO(una_csv,data_f+'fromHPRC/cauchy.avgvar00/dirUNIQUE/SISSO.out',3)

In [10]:
features_full=[]
for i in sK.features:
    for j in ['_avg','_red','_diff']:
        features_full.append(i+j)

In [11]:
features_full

['atomic_number_avg',
 'atomic_number_red',
 'atomic_number_diff',
 'atomic_radius_avg',
 'atomic_radius_red',
 'atomic_radius_diff',
 'en_pauling_avg',
 'en_pauling_red',
 'en_pauling_diff',
 'density_avg',
 'density_red',
 'density_diff',
 'electron_affinity_avg',
 'electron_affinity_red',
 'electron_affinity_diff',
 'en_allen_avg',
 'en_allen_red',
 'en_allen_diff',
 'heat_of_formation_avg',
 'heat_of_formation_red',
 'heat_of_formation_diff',
 'lattice_constant_avg',
 'lattice_constant_red',
 'lattice_constant_diff',
 'melting_point_avg',
 'melting_point_red',
 'melting_point_diff',
 'period_avg',
 'period_red',
 'period_diff',
 'valence_avg',
 'valence_red',
 'valence_diff']

In [12]:
features_latex=[
 'Z_{avg}',
 'Z_{red}',
 'Z_{diff}',
 'r_{a,avg}',
 'r_{a,red}',
 'r_{a,diff}',
 '\\chi_{Pauling,avg}',
 '\\chi_{Pauling,red}',
 '\\chi_{Pauling,diff}',
 '\\rho_{avg}',
 '\\rho_{red}',
 '\\rho_{diff}',
 'E_{ea,avg}',
 'E_{ea,red}',
 'E_{ea,diff}',
 '\chi_{Allen,avg}',
 '\chi_{Allen,red}',
 '\chi_{Allen,diff}',
 '\Delta H_{f,avg}',
 '\Delta H_{f,red}',
 '\Delta H_{f,diff}',
 'a_{avg}',
 'a_{red}',
 'a_{diff}',
 'T_{m,avg}',
 'T_{m,red}',
 'T_{m,diff}',
 'P_{avg}',
 'P_{red}',
 'P_{diff}',
 'VEC_{avg}',
 'VEC_{red}',
 'VEC_{diff}']

## Get all descriptors in latex text

In [13]:
print('C11')
sC11.print_formula_latex(features_full,features_latex)
print('C12')
sC12.print_formula_latex(features_full,features_latex)
print('C44')
sC44.print_formula_latex(features_full,features_latex)
print('K')
sK.print_formula_latex(features_full,features_latex)
print('G')
sG.print_formula_latex(features_full,features_latex)
print('E')
sE.print_formula_latex(features_full,features_latex)
print('v')
sv.print_formula_latex(features_full,features_latex)
print('GK')
sGK.print_formula_latex(features_full,features_latex)
print('cauchy')
scauchy.print_formula_latex(features_full,features_latex)

C11
$\begin{array}{c}
1.16\times 10 ^{1}+1.14\times 10 ^{0}((VEC_{avg})^3 sqrt(P_{avg}))\\
+4.77\times 10 ^{1}((VEC_{diff})^6)^6\\
-2.94\times 10 ^{-2}((a_{avg})^6 (VEC_{red}+VEC_{diff}))\\
\end{array}$
C12
$\begin{array}{c}
1.31\times 10 ^{2}+1.26\times 10 ^{0}((\rho_{avg} \chi_{Pauling,red}) (\chi_{Pauling,avg}+\chi_{Pauling,diff}))\\
-1.44\times 10 ^{2}((VEC_{diff})^6 (VEC_{red} E_{ea,red}))\\
+3.72\times 10 ^{2}((VEC_{diff})^3 (VEC_{diff} E_{ea,red}))\\
\end{array}$
C44
$\begin{array}{c}
-3.19\times 10 ^{2}+1.01\times 10 ^{2}(sqrt(P_{avg}) (VEC_{avg}/a_{avg}))\\
-1.87\times 10 ^{1}((VEC_{diff})^6)^6\\
-5.57\times 10 ^{-2}((E_{ea,avg}-E_{ea,diff})/(\chi_{Pauling,diff} a_{diff}))\\
\end{array}$
K
$\begin{array}{c}
-1.56\times 10 ^{2}+3.92\times 10 ^{0}(sqrt(T_{m,avg}) (VEC_{avg}/a_{avg}))\\
+1.95\times 10 ^{2}((T_{m,diff}/a_{diff})/(\rho_{avg})^6)\\
+6.92\times 10 ^{3}((VEC_{avg}+VEC_{diff})/(a_{avg} \Delta H_{f,avg}))\\
\end{array}$
G
$\begin{array}{c}
1.50\times 10 ^{2}-2.39\times 

## Random sampling in the standard simplex

In [14]:
elements = ['Mo','Nb','Ta','V','W']
d = len(elements)
n = int(1.0e5)
rand_seed = 98

np.random.seed(rand_seed)
X = np.random.rand(n,d)

X = -np.log(X)
print(np.size(X,0))
for i in range(np.size(X,0)):
    X[i,:] = X[i,:]/sum(X[i,:])
print(np.sum(X,axis=1))

100000
[ 1.  1.  1. ...,  1.  1.  1.]


## Save the sampling

In [16]:
pickle.dump(X,open(res_f+"randomX"+str(rand_seed)+str(n)+".p","wb"))

### multiprocess run of all the models

In [17]:
print('K')
tic=time.time()
pool=multiprocessing.Pool()
rK=pool.map(sK.ult_eq,X)
print(time.time()-tic)

print('G')
tic=time.time()
rG=pool.map(sG.ult_eq,X)
print(time.time()-tic)

print('E')
tic=time.time()
rE=pool.map(sE.ult_eq,X)
print(time.time()-tic)

print('C11')
tic=time.time()
rC11=pool.map(sC11.ult_eq,X)
print(time.time()-tic)

print('C12')
tic=time.time()
rC12=pool.map(sC12.ult_eq,X)
print(time.time()-tic)


print('C44')
tic=time.time()
rC44=pool.map(sC44.ult_eq,X)
print(time.time()-tic)

print('v')
tic=time.time()
rv=pool.map(sv.ult_eq,X)
print(time.time()-tic)

print('GK')
tic=time.time()
rGK=pool.map(sGK.ult_eq,X)
print(time.time()-tic)

print('cauchy')
tic=time.time()
rcauchy=pool.map(scauchy.ult_eq,X)
print(time.time()-tic)

K
136.99090695381165
G
136.28640747070312
E
137.10695576667786
C11
135.844144821167
C12
136.91946125030518
C44
137.27244687080383
v
137.3946750164032
GK
138.575665473938
cauchy
136.97410655021667


In [15]:
cols=elements+['K','G','E','C_11','C_12','C_44','v','GK','cauchy']

### assign results to a dataframe

In [17]:
df=pd.DataFrame(columns=cols)

In [18]:
for c,i in enumerate(cols):
    print(c)
    if c<5:
        df[i]=X[:,c]

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [19]:
df['K']=rK
df['G']=rG
df['E']=rE
df['C_11']=rC11
df['C_12']=rC12
df['C_44']=rC44
df['v']=rv
df['GK']=rGK
df['cauchy']=rcauchy

In [20]:
df

,Mo,Nb,Ta,V,W,K,G,E,C_11,C_12,C_44,v,GK,cauchy
0,0.103056,0.187540,0.396833,0.081924,0.230647,222.909599,66.497325,198.985877,310.794253,175.049213,75.330698,0.349457,0.330808,109.131875
1,0.280739,0.124630,0.122582,0.143194,0.328855,247.122241,91.400105,250.570967,375.406967,181.164155,94.334290,0.328351,0.385298,90.958751
2,0.298227,0.264769,0.020448,0.364459,0.052098,208.300894,57.240373,159.867360,298.566605,162.275669,55.430933,0.372911,0.284616,107.847816
3,0.005126,0.415579,0.170636,0.019339,0.389320,228.706649,70.465051,202.181051,322.253389,178.574845,78.524074,0.348332,0.324548,112.833329
4,0.069315,0.223607,0.004114,0.067489,0.635475,266.024399,109.306215,282.056206,408.556823,193.468228,110.389480,0.316539,0.403651,87.537470
5,0.499773,0.327282,0.029516,0.037568,0.105860,234.165523,84.508758,226.036196,359.885634,173.182701,82.213046,0.337824,0.368711,89.234172
6,0.054621,0.162526,0.176426,0.136894,0.469533,245.809639,86.517389,241.760955,360.727281,184.319935,93.552535,0.332699,0.367401,100.978743
7,0.008939,0.518274,0.219156,0.189171,0.064460,186.827340,38.289410,109.969376,241.885914,160.251832,39.788818,0.399102,0.214409,123.582521
8,0.002384,0.051291,0.132119,0.419438,0.394769,229.866856,71.983959,203.138411,327.427513,174.979045,76.769660,0.354010,0.320445,111.620330
9,0.299581,0.061208,0.133514,0.118385,0.387312,257.767014,100.496187,275.256641,398.950389,185.857165,104.264110,0.318100,0.411311,85.094505


In [21]:
df.to_csv(res_f+'results_SISSO_obj'+str(rand_seed)+str(n)+'.csv')